# Web scraping AirBnB

In [3]:
import os
import time
import pandas as pd
import urllib.parse as urlparse
from urllib.parse import parse_qs

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup

# Get current working directory
print(os.getcwd())

/Users/andreas/Nextcloud/cloud_atrip/Dokumente/Repos/applied-data-science


# Urls to scrape

In [4]:
# urlStringHome = 'https://www.airbnb.ch/'
# urlString = "https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=autocomplete_click&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt"
# urlStringFlat0to68 = "https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt&place_id=ChIJGaK-SZcLkEcRA9wf5_GNbuY&channel=EXPLORE&price_min=9&price_max=68"
# urlStringFlat69to84 = "https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt&place_id=ChIJGaK-SZcLkEcRA9wf5_GNbuY&channel=EXPLORE&price_min=69&price_max=84"
# urlStringFlat85to100 = "https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt&place_id=ChIJGaK-SZcLkEcRA9wf5_GNbuY&channel=EXPLORE&price_min=85&price_max=100"
# urlStringFlat101to114 = "https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt&place_id=ChIJGaK-SZcLkEcRA9wf5_GNbuY&channel=EXPLORE&price_min=101&price_max=114"
# urlStringFlat115to132 = "https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt&place_id=ChIJGaK-SZcLkEcRA9wf5_GNbuY&channel=EXPLORE&price_min=115&price_max=132"
# urlStringFlat133to150 = "https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt&place_id=ChIJGaK-SZcLkEcRA9wf5_GNbuY&channel=EXPLORE&price_min=133&price_max=150"
# urlStringFlat151to173 = "https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt&place_id=ChIJGaK-SZcLkEcRA9wf5_GNbuY&channel=EXPLORE&price_min=151&price_max=173"
# urlStringFlat174to202 = "https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt&place_id=ChIJGaK-SZcLkEcRA9wf5_GNbuY&channel=EXPLORE&price_min=151&price_max=173"
# urlStringFlat203to245 = "https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt&place_id=ChIJGaK-SZcLkEcRA9wf5_GNbuY&channel=EXPLORE&price_min=203&price_max=245"
# urlStringFlat246to316 = "https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt&place_id=ChIJGaK-SZcLkEcRA9wf5_GNbuY&channel=EXPLORE&price_min=246&price_max=316"
# urlStringFlat317to500 = "https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt&place_id=ChIJGaK-SZcLkEcRA9wf5_GNbuY&channel=EXPLORE&price_min=317&price_max=500"
# urlStringFlat501to1000 = "https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt&place_id=ChIJGaK-SZcLkEcRA9wf5_GNbuY&channel=EXPLORE&price_min=501&price_max=1000"
# define price range for scraping, place etc is already definied in buildURL func
url_range = [[0,68],[69,84],[85,100],[101,114],[114,132],[133,150],[151,173],[174,202],[203,245],[246,316],[317,500],[501,1000]]

In [5]:
def buildURL(p_range):
    builded_urls = []
    for p in p_range:
        url = f"https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt&place_id=ChIJGaK-SZcLkEcRA9wf5_GNbuY&channel=EXPLORE&price_min={p[0]}&price_max={p[1]}"
        builded_urls.append(url)
    return builded_urls

## Fields to scrape from detail apartment page

* Preis
* Standort
* Gesamt Rating
  * Alle Subfelder Sauberkeit, Kommunikation etc.
* Anz. Gäste
* Anz. Schlafzimmer 
* Anz. Better
* Anz. Badezimmer 
* Ameneties
* Koordinaten Wohnung
* Anz. Bewertungen
* Kommentare

## Helper functions for scraping

In [6]:
# helper func to scrape comments
def scrape_comments(driver):
    comments = []
    XPATH_COMMENTS_BTN = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[4]/div/div/div/div[2]/section/div[4]/button"
    XPATH_COMMENTS_BTN_CLOSE = "/html/body/div[9]/section/div/div/div[2]/div/div[1]/button"
    XPATH_COMMENTS_CLASS = "span[class='ll4r2nl dir dir-ltr']"
    XPATH_COMMENTS_ON_SITE = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[4]/div/div/div/div[2]/section/div[3]/div/div/*"

    try:
        WebDriverWait(driver, 4).until(EC.visibility_of_element_located(
            (By.XPATH, "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[4]/div/div/div/div[2]")))
    except:
        print("could not load comments title")
        return comments

    try:
        if driver.find_elements(By.XPATH, XPATH_COMMENTS_BTN):
            driver.find_element(By.XPATH, XPATH_COMMENTS_BTN).click()
            resu = driver.find_elements(By.CSS_SELECTOR, XPATH_COMMENTS_CLASS)
            for comm in resu:
                if comm.text:
                    comments.append(comm.text)
            driver.find_element(By.XPATH, XPATH_COMMENTS_BTN_CLOSE).click()

        elif not driver.find_elements(By.XPATH, XPATH_COMMENTS_BTN) and len(driver.find_elements(By.XPATH, XPATH_COMMENTS_ON_SITE)) > 0:
            comments_on_page = driver.find_elements(
                By.XPATH, XPATH_COMMENTS_ON_SITE)
            for comm_on in comments_on_page:
                if comm_on.text:
                    temp_comm_on = comm_on.text.split("\n", maxsplit=2)[2:]
                    comments.append(temp_comm_on[0])

        else:
            return comments
    
    except:
        print("could not process comments")
        return comments

    return comments


In [7]:
# helper func to scrape coordinates
def scrape_coordinates(driver):
    lat = 0.0
    lng = 0.0

    try:
        element_to_be_present_co = WebDriverWait(driver, 4).until(EC.visibility_of_element_located((By.XPATH, "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[5]/div/div/div/div[2]/section/div[1]/div/h2")))
        driver.execute_script("arguments[0].scrollIntoView();", element_to_be_present_co)

        time.sleep(1)

        coordinates_elem = WebDriverWait(driver, 4).until(EC.visibility_of_element_located((By.XPATH, "//*[contains(@href, 'https://maps.google.com/maps?ll=')]")))
        coordinates_href = coordinates_elem.get_attribute('href')

        parsed_href = urlparse.urlparse(coordinates_href)
        coordinates = parse_qs(parsed_href.query)['ll'][0].split(',')
        lat = float(coordinates[0])
        lng = float(coordinates[1])
    except:
        print("Could not process coordinates")

    return lat, lng


In [8]:
# helper func to scrape ameneties
def scrape_ameneties(driver):

    ameneties_list = []

    try:
        SHOW_AMENETIES_CLASS = "button[class='l1j9v1wn b65jmrv v7aged4 dir dir-ltr']"
        driver.find_element(By.CSS_SELECTOR, SHOW_AMENETIES_CLASS).click()

        time.sleep(1)

        element_present_ame = EC.presence_of_element_located(
            (By.XPATH, "/html/body/div[9]/section/div/div/div[2]/div/div[3]"))
        WebDriverWait(driver, 5).until(element_present_ame)

        soup = BeautifulSoup(driver.page_source, features='html.parser')
        ameneties = soup.find_all("div", {"class": "t1dx2edb dir dir-ltr"})

        for a in ameneties:
            if "</span>" not in a.decode_contents():
                ameneties_list.append(a.decode_contents())

    except:
        print("Could not process ameneties")

    try:
        driver.find_element(
            By.XPATH, "/html/body/div[9]/section/div/div/div[2]/div/div[1]/button").click()
    except:
        print("Could not close ameneties window")

    return ameneties_list


In [9]:
# scrapping logic for detail page apartment
def scrape_apartment(nr_of_flat, driver):

    flat = None

    XPATH_cleanliness = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[4]/div/div/div/div[2]/section/div[2]/div/div/div[1]/div/div/div[2]/span"
    XPATH_communication = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[4]/div/div/div/div[2]/section/div[2]/div/div/div[3]/div/div/div[2]/span"
    XPATH_checkin = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[4]/div/div/div/div[2]/section/div[2]/div/div/div[5]/div/div/div[2]/span"
    XPATH_valueformoney = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[4]/div/div/div/div[2]/section/div[2]/div/div/div[6]/div/div/div[2]/span"
    XPATH_location = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[4]/div/div/div/div[2]/section/div[2]/div/div/div[4]/div/div/div[2]/span"
    XPATH_data_accuracy = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[4]/div/div/div/div[2]/section/div[2]/div/div/div[2]/div/div/div[2]/span"
    rating_cleanliness = ""
    rating_communication = ""
    rating_checkin = ""
    rating_valueformoney = ""
    rating_location = ""
    rating_data_accuracy = ""

    XPATH_APARETMENT_NAME = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[1]/div[1]/div/div/div/div/section/div[1]/span/h1"
    apartment_name = ""

    XPATH_PRICE = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[3]/div/div[2]/div/div/div[1]/div/div/div/div/div/div/div[1]/div[1]/div[1]/div/span/div/span[1]"
    apartment_price = ""
    XPATH_PRICE_REDUCED = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[3]/div/div[2]/div/div/div[1]/div/div/div/div/div/div/div[1]/div[1]/div[1]/div/span/div/span[2]"
    apartment_price_reduced = ""

    XPATH_PLACE = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[1]/div[1]/div/div/div/div/section/div[2]/div[1]/span[3]/button/span"
    XPATH_PLACE_2 = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[1]/div[1]/div/div/div/div/section/div[2]/div[1]/span[5]/button/span"
    apartment_place = ""

    XPATH_RATING = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[1]/div[1]/div/div/div/div/section/div[2]/div[1]/span[1]/span[2]"
    rating = ""

    XPATH_GUESTS = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[3]/div/div[1]/div/div[1]/div/div/section/div/div/div/div[1]/ol/li[1]/span[1]"
    guests = ""
    XPATH_BEDROOMS = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[3]/div/div[1]/div/div[1]/div/div/section/div/div/div/div[1]/ol/li[2]/span[2]"
    bedrooms = ""
    XPATH_BEDS = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[3]/div/div[1]/div/div[1]/div/div/section/div/div/div/div[1]/ol/li[3]/span[2]"
    beds = ""
    XPATH_BATHROOMS = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[3]/div/div[1]/div/div[1]/div/div/section/div/div/div/div[1]/ol/li[4]/span[2]"
    bathrooms = ""

    XPATH_NR_OF_RATINGS = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[1]/div[1]/div/div/div/div/section/div[2]/div[1]/span[1]/span[3]/button"
    nr_of_ratings = ""

    # check if page title is present
    try:
        element_present = EC.presence_of_element_located(
            (By.XPATH, "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[1]/div[1]/div/div/div/div/section/div[1]/span/h1"))
        WebDriverWait(driver, 8).until(element_present)
    except TimeoutException:
        print("Timed out waiting for page to load")
        return

    try:
        if driver.find_elements(By.XPATH, XPATH_PLACE):
            apartment_place = WebDriverWait(driver, 3).until(
                EC.visibility_of_element_located((By.XPATH, XPATH_PLACE))).text
            if "Zürich" not in apartment_place:
                return flat
        else:
            apartment_place = WebDriverWait(driver, 3).until(
                EC.visibility_of_element_located((By.XPATH, XPATH_PLACE_2))).text
            if "Zürich" not in apartment_place:
                return flat
    except:
        pass

    try:
        apartment_name = WebDriverWait(driver, 3).until(
            EC.visibility_of_element_located((By.XPATH, XPATH_APARETMENT_NAME))).text
        nr_of_ratings = WebDriverWait(driver, 4).until(
            EC.visibility_of_element_located((By.XPATH, XPATH_NR_OF_RATINGS))).text
    except:
        pass

    try:
        apartment_price = WebDriverWait(driver, 4).until(
            EC.visibility_of_element_located((By.XPATH, XPATH_PRICE))).text

        if driver.find_elements(By.XPATH, XPATH_PRICE_REDUCED):
            apartment_price_reduced = driver.find_element(
                By.XPATH, XPATH_PRICE_REDUCED).text
            if "CHF" not in apartment_price_reduced:
                apartment_price_reduced = ""
    except:
        pass

    try:
        rating = driver.find_element(By.XPATH, XPATH_RATING).text
        rating_checkin = driver.find_element(By.XPATH, XPATH_checkin).text
        rating_cleanliness = driver.find_element(
            By.XPATH, XPATH_cleanliness).text
        rating_communication = driver.find_element(
            By.XPATH, XPATH_communication).text
        rating_data_accuracy = driver.find_element(
            By.XPATH, XPATH_data_accuracy).text
        rating_location = driver.find_element(By.XPATH, XPATH_location).text
        rating_valueformoney = driver.find_element(
            By.XPATH, XPATH_valueformoney).text
    except:
        pass

    try:
        guests = driver.find_element(By.XPATH, XPATH_GUESTS).text
        bedrooms = driver.find_element(By.XPATH, XPATH_BEDROOMS).text
        beds = driver.find_element(By.XPATH, XPATH_BEDS).text
        bathrooms = driver.find_element(By.XPATH, XPATH_BATHROOMS).text
    except:
        pass

    apartment_ameneties = scrape_ameneties(driver)
    comments = scrape_comments(driver)
    lat, lng = scrape_coordinates(driver)

    flat = {'name': [apartment_name], 'price': [apartment_price], 'price_reduced': [apartment_price_reduced], 'place': [apartment_place], 'guests': [guests], 'bedrooms': [bedrooms], 'beds': [beds], 'bathrooms': [bathrooms], 'lat': [lat], 'lng': [lng], 'nr_of_ratings': [nr_of_ratings], 'rating_overall': [rating], 'rating_valueformoney': [rating_valueformoney],
            'rating_location': [rating_location], 'rating_checkin': [rating_checkin], 'rating_communication': [rating_communication], 'rating_cleanliness': [rating_cleanliness], 'rating_data_accuracy': [rating_data_accuracy], 'comments': [comments], 'list_ameneties': [apartment_ameneties]}

    #print(nr_of_flat, flat)

    return flat


In [10]:
def close_tab(driver):
    try:
        # close tab
        parent = driver.window_handles[0]
        chld = driver.window_handles[1]
        driver.switch_to.window(chld)
        driver.close()
        time.sleep(0.5)
        parent = driver.window_handles[0]
        driver.switch_to.window(parent)
    except:
        print("could not close tab")

In [11]:
# helper func to open each aparement in sepeerate tab
def handle_apartments_tabs(nr_apartments, driver):
    # Click on each apartment which is opened in seperate tab
    apartments = pd.DataFrame()
    for nr in range(1, nr_apartments):
        xpath = f"/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div/div/div[2]/main/div[2]/div[2]/div/div/div/div/div/div[{nr}]"
        driver.find_element(By.XPATH, xpath).click()

        try:
            # switch tab
            #get old tab
            parent = driver.window_handles[0]
            time.sleep(3)
            #get new tab
            chld = driver.window_handles[1]
            #switch to new tab
            driver.switch_to.window(chld)
        except:
            print("could not switch tab")
            continue

        if nr == 1:
            # first time tap is opened selenium needs some time to get running properly...
            time.sleep(3)
            try:
                # close translate banner
                transl = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[9]/section/div/div/div[2]/div/div[1]/button"))) 
                transl.click()
            except:
                pass
        
        apartment_scraped = scrape_apartment(nr, driver)

        if apartment_scraped == None:
            close_tab(driver)
            continue

        apartments = pd.concat([apartments, pd.DataFrame(apartment_scraped)])
        close_tab(driver)
    
    return apartments
    

# Run scrape logic

In [23]:
url_to_scrape = buildURL(url_range)
url_to_scrape

['https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt&place_id=ChIJGaK-SZcLkEcRA9wf5_GNbuY&channel=EXPLORE&price_min=0&price_max=68',
 'https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&price_filter_num_nights=5&adults=1&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change&query=Z%C3%BCrich%2C%20Schweiz&room_types%5B%5D=Entire%20home%2Fapt&place_id=ChIJGaK-SZcLkEcRA9wf5_GNbuY&channel=EXPLORE&price_min=69&price_max=84',
 'https://www.airbnb.ch/s/Z%C3%BCrich--Schweiz/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B

In [ ]:
# run headless
# Set options for Chrome-Driver (headless mode)
opts = Options()
#opts.add_argument("--window-size=1200,800")
opts.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)

# Website to scrape
driver.get(url_to_scrape[3])

# Get current url
print(driver.current_url)

# Return the full page HTML code
# print(driver.page_source)

# Gets the page's title
print(driver.title)

# Close driver
# driver.close()

In [ ]:
# wait till cookie banner pops up
try:
    # close cookie banner
    cookie = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "/html[@class='js-focus-visible']/body[@class='with-new-header']/div[5]/div/div/div/div[1]/div[3]/section/div[2]/div[2]/button"))) 
    cookie.click()
except:
    pass

In [ ]:
# get nr of pages with aparements
nr_pages = len(driver.find_elements(By.XPATH, "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div/div/div[2]/main/div[2]/div[3]/div/div/nav/div/*"))
# subtract forward backwards buttons
nr_pages = nr_pages - 2

In [ ]:
nr_pages

In [ ]:
# get entries of apparements on page
nr_apartments = len(driver.find_elements(By.XPATH, "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div/div/div[2]/main/div[2]/div[2]/div/div/div/div/div/div"))

In [ ]:
nr_apartments

In [ ]:
# all entries
scraped_apartments = pd.DataFrame()
XPATH_NEXT_PAGE = "//a[@aria-label='Weiter']"
XPATH_WAIT_TILL_ELEM_VISIBLE = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div/div/div[2]/main/div[2]/div[2]/div/div/div/div/div/div[1]"

for nr_p in range(1, nr_pages):
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, XPATH_WAIT_TILL_ELEM_VISIBLE)))
    temp_pd = handle_apartments_tabs(nr_apartments + 1, driver)
    scraped_apartments = pd.concat([scraped_apartments, temp_pd])
    driver.find_element(By.XPATH, XPATH_NEXT_PAGE).click()

In [ ]:
# limited entries for tests 
scraped_apartments = pd.DataFrame()
XPATH_NEXT_PAGE = "//a[@aria-label='Weiter']"
XPATH_WAIT_TILL_ELEM_VISIBLE = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div/div/div[2]/main/div[2]/div[2]/div/div/div/div/div/div[1]"

for nr_p in range(1, nr_pages):
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, XPATH_WAIT_TILL_ELEM_VISIBLE)))
    temp_pd = handle_apartments_tabs(4, driver)
    scraped_apartments = pd.concat([scraped_apartments, temp_pd])
    driver.find_element(By.XPATH, XPATH_NEXT_PAGE).click()

In [ ]:
scraped_apartments.head()

In [ ]:
# close all tabs and window
for handles in driver.window_handles:
    parent = handles
    driver.switch_to.window(parent)
    driver.close()

In [ ]:
scraped_apartments.to_csv("./airbnb_apartments_scraped.csv", encoding="utf-8", index=False, header=True, errors="replace")

# Run scrape logic in parallel

In [31]:
from pandas import DataFrame

def run_scraping(airbnb_url_to_scrape) -> DataFrame:
    opts = Options()
    #opts.add_argument("--window-size=1200,800")
    opts.add_argument("--headless")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)
    driver.get(airbnb_url_to_scrape)

    time.sleep(5)

    # wait till cookie banner pops up
    try:
        # close cookie banner
        cookie = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "/html[@class='js-focus-visible']/body[@class='with-new-header']/div[5]/div/div/div/div[1]/div[3]/section/div[2]/div[2]/button"))) 
        cookie.click()
    except:
        pass

    # get nr of pages with aparements
    nr_pages = len(driver.find_elements(By.XPATH, "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div/div/div[2]/main/div[2]/div[3]/div/div/nav/div/*"))
    # subtract forward backwards buttons
    nr_pages = nr_pages - 2

    nr_apartments = len(driver.find_elements(By.XPATH, "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div/div/div[2]/main/div[2]/div[2]/div/div/div/div/div/div"))

    # limited entries for tests 
    scraped_apartments = pd.DataFrame()
    XPATH_NEXT_PAGE = "//a[@aria-label='Weiter']"
    XPATH_WAIT_TILL_ELEM_VISIBLE = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div/div/div[2]/main/div[2]/div[2]/div/div/div/div/div/div[1]"

    for nr_p in range(1, nr_pages):
        WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, XPATH_WAIT_TILL_ELEM_VISIBLE)))
        temp_pd = handle_apartments_tabs(nr_apartments + 1, driver)
        #temp_pd = handle_apartments_tabs(nr_apartments + 1, driver)
        scraped_apartments = pd.concat([scraped_apartments, temp_pd])
        driver.find_element(By.XPATH, XPATH_NEXT_PAGE).click()
    
    # close all tabs and window
    for handles in driver.window_handles:
        parent = handles
        driver.switch_to.window(parent)
        driver.close()

    return scraped_apartments

In [32]:
import time
import multiprocessing as mp
import multiprocessing_func

scraped_apartments = pd.DataFrame()
url_to_scrape = buildURL(url_range)

with mp.Pool(4) as pool:
    result = pool.map(multiprocessing_func.run_scraping, url_to_scrape)

for res in result:
    scraped_apartments = pd.concat([scraped_apartments, res])

scraped_apartments.to_csv("./airbnb_apartments_scraped.csv", encoding="utf-8", index=False, header=True, errors="replace")

could not switch tab
could not switch tab
could not switch tab
could not switch tab
could not switch tab
could not switch tab
could not switch tab
could not switch tab
could not switch tab
could not switch tab
could not switch tab
could not switch tab
could not switch tab
could not switch tab
could not switch tab
could not switch tab
could not switch tab
could not switch tab
could not switch tab
could not switch tab
could not switch tab
could not switch tab
could not switch tab
could not switch tab
could not switch tab
could not switch tab
could not switch tab
could not switch tab
could not switch tab
could not switch tab
could not switch tab
could not switch tab
could not switch tab
could not switch tab
